In [5]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, RepeatedKFold, cross_val_score
from sklearn.metrics import accuracy_score, log_loss
from joblib import dump, load
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 14.9 ms


In [160]:
training_data = pd.read_csv("training_data_full.csv")

time: 4.07 s


In [6]:
training_data.head()

,good_claim,proceeding_number,patent_id,detailed_description,days_between_grant_and_filing,description_word_count,fk_score,fig_counts,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn,uuid,classification_level,section,ipc_class,subclass,main_group,subgroup,symbol_position,classification_value,classification_status,classification_data_source,action_date,ipc_version_indicator,sequence
0,0,PGR2018-00008,9597594,"DESCRIPTION Hereinafter, with reference to the...",231,13494,13.314722,69,9597594,utility,9597594,US,2017-03-21,Provided is a method for controlling a compute...,"Computer control method, control program and c...",B2,20.0,ipg170321.xml,0.0,5q202ek2qg21dsm3ynl44zzor,A,A,63,F,13,822,L,I,B,H,2017-03-21,2014-01-01,2
1,1,IPR2018-01566,8367337,"As discussed herein, the present invention pro...",2023,7893,19.433383,8,8367337,utility,8367337,US,2013-02-05,The present invention provides improved tests ...,Detection of methicillin-resistant Staphylococ...,B2,47.0,ipg130205.xml,0.0,b2jrwkfubjdln0d820f44kyhb,A,C,7,H,21,04,L,I,B,H,2013-02-05,2006-01-01,2
2,0,IPR2019-00015,8409052,DESCRIPTION OF THE PREFERRED EMBODIMENT An exe...,2038,4224,14.952263,17,8409052,utility,8409052,US,2013-04-02,Provided is a starting method for hybrid elect...,Starting method for hybrid electric vehicle an...,B2,16.0,ipg130402.xml,0.0,bp4w0r1ahp4onk16nzuxx2tss,A,B,60,W,10,24,L,I,B,H,2013-04-02,2006-01-01,1
3,1,IPR2018-00159,9023979,DESCRIPTION OF SPECIFIC EMBODIMENTS Hereinafte...,920,9089,20.703933,7,9023979,utility,9023979,US,2015-05-05,This invention relates to a method of preparin...,Precise control of molecular weight and chain ...,B2,4.0,ipg150505.xml,0.0,4ayfcysyb77vs9fz7toqhvi5c,A,C,8,G,65,04,L,N,B,H,2015-05-05,2006-01-01,7
4,1,IPR2017-02108,8508751,DETAILED DESCRIPTION OF THE EMBODIMENTS With r...,1494,2446,15.367966,11,8508751,utility,8508751,US,2013-08-13,Disclosed is a method of acquiring an image of...,Capturing real-time video with zooming capabil...,B1,20.0,ipg130813.xml,0.0,z2uoy0oz3akvjirh28zms34tp,A,G,6,F,3,12,F,I,B,H,2013-08-13,2006-01-01,0


time: 75.5 ms


In [161]:
categories = ['kind', 'section']
training_data_plus_categories = pd.get_dummies(training_data, columns=categories)

time: 62.2 ms


In [162]:
category_names = list(training_data_plus_categories.columns[33-len(categories):])

time: 2.64 ms


In [163]:
category_names

['kind_A',
 'kind_B1',
 'kind_B2',
 'kind_E',
 'kind_E1',
 'section_A',
 'section_B',
 'section_C',
 'section_D',
 'section_E',
 'section_F',
 'section_G',
 'section_H',
 'section_M']

time: 4.96 ms


In [164]:
X = training_data_plus_categories[['days_between_grant_and_filing', 'description_word_count', 'fk_score', 'fig_counts', 'ipc_class'] + category_names]
y = training_data_plus_categories['good_claim']

time: 26 ms


In [169]:
model = GradientBoostingClassifier(learning_rate=.1, n_estimators=100, max_depth=7, subsample=1, min_samples_split=2, random_state=33)
# model = RandomForestClassifier(n_estimators=50, random_state=33, max_depth=3, min_samples_split=2, max_features="sqrt")
# model = LogisticRegression()
# model = DecisionTreeClassifier()
# model = KNeighborsClassifier(n_neighbors=100, n_jobs=-1)
cv_results = cross_validate(model, X, y, cv=RepeatedKFold(n_splits=7, n_repeats=3, random_state=7), n_jobs=-1, return_train_score=True, 
                            scoring=['accuracy', 'neg_log_loss'])

time: 27.5 s


In [95]:
# Days, word count, fk score, fig counts, kinds, section, ipc_class
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.7712890839627917
Log Loss: 0.5058363743721214
time: 3.91 ms


In [92]:
# Days, word count, fk score, fig counts, kinds, section
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.7655424249643261
Log Loss: 0.5083122413183988
time: 2.17 ms


In [86]:
# Days, word count, fk score, fig counts, kinds
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.7614007140410697
Log Loss: 0.5151745028662464
time: 2.86 ms


In [80]:
# Days, word count, fk score, fig counts -- max_depth 7
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.7605999645466016
Log Loss: 0.5223985236744204
time: 4.51 ms


In [36]:
# Days, word count, fk score, fig counts -- max_depth 5  --- model iteration 1
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.7422466545868435
Log Loss: 0.5434979089880754
time: 3.76 ms


In [10]:
### Naive Model
print("Naive Accuracy: {}".format(y.mean()))
print("Naive Log Loss: {}".format(log_loss(y, [y.mean()]*len(y))))

Naive Accuracy: 0.7109751575824991
Naive Log Loss: 0.6012762313064095
time: 451 ms


In [11]:
model.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=33,
              subsample=1, tol=0.0001, validation_fraction=0.1, verbose=0,
              warm_start=False)

time: 2.66 s


In [13]:
dump(model, 'patent_GBC.joblib') 

['patent_GBC.joblib']

time: 222 ms
